In [4]:
!pip install tensorflow --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



  Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl (376.9 MB)
  Using cached protobuf-4.25.3-cp39-cp39-win_amd64.whl (413 kB)
  Using cached grpcio-1.62.1-cp39-cp39-win_amd64.whl (3.8 MB)
  Using cached keras-3.1.1-py3-none-any.whl (1.1 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached h5py-3.10.0-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl (26.4 MB)
  Using cached ml_dtypes-0.3.2-cp39-cp39-win_amd64.whl (127 k

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

C:\Users\user\Downloads\python\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load the data
train_data = pd.read_csv("C:/Users/user/OneDrive/Desktop/train.csv", encoding = 'latin1')
test_data = pd.read_csv('C:/Users/user/OneDrive/Desktop/test.csv', encoding = 'latin1')

In [3]:
# Preprocessing
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

tokenizer_article = Tokenizer(oov_token='<OOV>')
tokenizer_event = Tokenizer(oov_token='<OOV>')

# Remove stop words
def remove_stop_words(text):
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(tokens)

train_data['article'] = train_data['article'].apply(remove_stop_words)
# Fill missing values with an empty string
train_data['event_description'] = train_data['event_description'].fillna('')

# Apply the remove_stop_words function
train_data['event_description'] = train_data['event_description'].apply(remove_stop_words)
test_data['article'] = test_data['article'].apply(remove_stop_words)

tokenizer_article.fit_on_texts(train_data['article'])
tokenizer_event.fit_on_texts(train_data['event_description'])

vocab_size_article = len(tokenizer_article.word_index) + 1
vocab_size_event = len(tokenizer_event.word_index) + 1

max_article_length = max([len(article.split()) for article in train_data['article']])
max_event_length = max([len(event.split()) for event in train_data['event_description']])

# Check tokenized word indices for articles
print("Word indices for articles:")
print(tokenizer_article.word_index)

# Check tokenized word indices for event descriptions
print("\nWord indices for event descriptions:")
print(tokenizer_event.word_index)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Word indices for articles:
{'<OOV>': 1, 'market': 2, 'cisco': 3, 'growth': 4, 'ai': 5, 'systems': 6, 'cloud': 7, 'security': 8, 'inc': 9, '1': 10, 'data': 11, '0': 12, 'solutions': 13, 'company': 14, 'services': 15, 'year': 16, '2023': 17, 'industry': 18, 'companies': 19, 'million': 20, 'digital': 21, 'global': 22, 'business': 23, 'revenue': 24, '2': 25, 'technology': 26, 'research': 27, 'report': 28, 'technologies': 29, '2024': 30, '3': 31, 'stock': 32, '4': 33, 'infrastructure': 34, 'us': 35, 'content': 36, 'key': 37, 'price': 38, 'software': 39, 'new': 40, 'billion': 41, 'access': 42, 'based': 43, '5': 44, '2022': 45, 'cyber': 46, 'businesses': 47, 'also': 48, 'analysis': 49, 'america': 50, 'high': 51, 'forecast': 52, 's': 53, 'across': 54, 'management': 55, '7': 56, 'insights': 57, 'time': 58, 'networks': 59, 'com': 60, 'demand': 61, 'like': 62, 'dow': 63, 'networking': 64, 'service': 65, 'period': 66, 'investors': 67, 'quarter': 68, 'euros': 69, 'platform': 70, 'nvidia': 71, 'one'

In [4]:
print(test_data.columns)


Index(['article', 'event_description'], dtype='object')


In [5]:
#Prepare data for training
def prepare_data(data, tokenizer_article, tokenizer_event, max_article_length, max_event_length):
    encoder_input_data = tokenizer_article.texts_to_sequences(data['article'])
    decoder_input_data = tokenizer_event.texts_to_sequences(data['event_description'])

    encoder_input_data = pad_sequences(encoder_input_data, maxlen=max_article_length, padding='post')
    decoder_input_data = pad_sequences(decoder_input_data, maxlen=max_event_length, padding='post')

    decoder_target_data = np.zeros((len(data), max_event_length, vocab_size_event), dtype='float32')
    for i, seq in enumerate(decoder_input_data):
        for j, token_id in enumerate(seq):
            if j > 0:
                decoder_target_data[i][j-1][token_id] = 1.0

    return encoder_input_data, decoder_input_data, decoder_target_data

encoder_input_train, decoder_input_train, decoder_target_train = prepare_data(train_data, tokenizer_article, tokenizer_event, max_article_length, max_event_length)
encoder_input_test, decoder_input_test, decoder_target_test = prepare_data(test_data, tokenizer_article, tokenizer_event, max_article_length, max_event_length)


In [6]:
# Define the model
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size_article, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size_event, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_event, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [7]:
# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([encoder_input_train, decoder_input_train], decoder_target_train, batch_size=64, epochs=5, validation_split=0.2)

Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 2.8716e-04 - loss: 8.0283 - val_accuracy: 0.0022 - val_loss: 8.0274
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 7s/step - accuracy: 0.0051 - loss: 8.0247 - val_accuracy: 0.0078 - val_loss: 8.0251
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 34s 8s/step - accuracy: 0.0120 - loss: 8.0210 - val_accuracy: 0.0072 - val_loss: 8.0216
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 9s/step - accuracy: 0.0097 - loss: 8.0130 - val_accuracy: 0.0056 - val_loss: 8.0095
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 9s/step - accuracy: 0.0066 - loss: 7.9863 - val_accuracy: 0.0050 - val_loss: 7.9163


In [8]:
# Generate predictions on the test data
decoder_target_pred = model.predict([encoder_input_test, decoder_input_test])

# Convert predictions to text
def sequences_to_texts(sequences, tokenizer):
    texts = []
    for sequence in sequences:
        text = tokenizer.sequences_to_texts([sequence])[0]
        texts.append(text)
    return texts

decoder_target_pred_texts = sequences_to_texts(np.argmax(decoder_target_pred, axis=2), tokenizer_event)

# Analyze predicted words and extract sentences containing them
predicted_sentences = []
for i, text in enumerate(decoder_target_pred_texts):
    predicted_word = text.split()[0]  # Extract the first word from the predicted text
    article_sentences = test_data['article'].iloc[i].split('.')  # Split the article into sentences
    # Find sentences containing the predicted word
    relevant_sentences = [sentence.strip() for sentence in article_sentences if predicted_word in sentence]
    predicted_sentences.append(relevant_sentences)

# Print predicted sentences
print("Predicted Sentences:")
for i, sentences in enumerate(predicted_sentences):
    print("Article:", test_data['article'].iloc[i])
    print("Predicted Sentences:")
    for sentence in sentences:
        print(sentence)
    print("Actual Event Description:")
    print(test_data['event_description'].iloc[i])
    print("\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Sentences:
Article: Pioneering tech expert former Apple Inc. executive Saori Casey reportedly set assume role chief financial officer smart speaker manufacturer Sonos Inc. SONO, marking another significant executive departure Apple. Happened: Casey succeed Eddie Lazarus CFO Jan. 22. Meanwhile, Lazarus shift gears become chief strategy officer continuing role chief legal officer, according statement made developer manufacturer audio products. Casey impressive track record Apple, helmed vice president finance position decade. managed financial planning, forecasting, investor relations Apple, reported Bloomberg. Casey's transition comes Sonos venturing new markets headphones, following slowdown primary smart audio equipment sector. Matters: Casey's move Sonos latest series executive departures Apple. Tang Tan, Apple iPhone design executive, set leave tech giant February join groundbreaking AI project. Previously reported Tan collaborate Jony I